In [45]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

# ベースURL
base_url = "https://www.jalan.net/ikisaki/map/tokyo/"
response = requests.get(base_url)

# メタタグからエンコーディングを取得して設定
soup = BeautifulSoup(response.content, "html.parser")
meta_tag = soup.find("meta", {"charset": True})

if meta_tag and meta_tag.get("charset"):
    response.encoding = meta_tag.get("charset")
else:
    response.encoding = response.apparent_encoding

# 再度パース
soup = BeautifulSoup(response.text, "html.parser")

# 'map-right-inner' を持つ div を取得
map_right_inner_div = soup.find("div", class_="map-right-inner")
if map_right_inner_div:
    a_tags = map_right_inner_div.find_all("a", class_="sub")
    print(f"発見した a タグの数: {len(a_tags)}")
else:
    print("対象の div が見つかりませんでした。")
    a_tags = []

# URLを収集
urls = [urljoin(base_url, a['href']) for a in a_tags if 'href' in a.attrs]

# 特定の文字化けを修正する関数
def fix_encoding(text):
    try:
        return text.encode('latin1').decode('utf-8')
    except (UnicodeEncodeError, UnicodeDecodeError):
        return text  # 修正できない場合そのまま返す

# ページ内の全てのホテル情報を取得
def get_hotel_data_from_page(url):
    response = requests.get(url)

    # メタタグでエンコーディングを確認して設定
    soup = BeautifulSoup(response.content, "html.parser")
    meta_tag = soup.find("meta", {"charset": True})
    if meta_tag and meta_tag.get("charset"):
        response.encoding = meta_tag.get("charset")
    else:
        response.encoding = response.apparent_encoding

    # 再度HTMLをパース
    soup = BeautifulSoup(response.text, "html.parser")

    # ホテル情報を取得
    hotel_divs = soup.find_all("div", class_="p-yadoCassette__body p-searchResultItem__body")
    hotels = []

    for hotel_div in hotel_divs:
        # ホテル名
        h2_tag = hotel_div.find("h2", class_="p-searchResultItem__facilityName")
        raw_hotel_name = h2_tag.get_text(strip=True) if h2_tag else "ホテル名なし"
        hotel_name = fix_encoding(raw_hotel_name)

        # 詳細ページURL
        a_tag = hotel_div.find("a", class_="jlnpc-yadoCassette__link s16_00 fb")
        detail_url = a_tag['data-href'] if a_tag and 'data-href' in a_tag.attrs else "URLなし"

        # アクセス情報
        access_tag = hotel_div.find("dd", class_="p-searchResultItem__accessValue")
        access_info = access_tag.get_text(strip=True) if access_tag else "アクセス情報なし"

        # 価格情報
        price_tag = hotel_div.find("span", class_="p-searchResultItem__lowestPriceValue")
        price_info = price_tag.get_text(strip=True) if price_tag else "価格情報なし"

        # 一人あたり価格
        unit_price_tag = hotel_div.find("span", class_="p-searchResultItem__lowestUnitPrice")
        unit_price_info = unit_price_tag.get_text(strip=True) if unit_price_tag else "一人あたり価格情報なし"

        # ホテルの評価
        rating_tag = hotel_div.find("span", class_="p-searchResultItem__summaryaverage-num")
        rating_info = rating_tag.get_text(strip=True) if rating_tag else "評価なし"

        # レビュー数
        review_count_tag = hotel_div.find("span", class_="ji ji-bubble-outline")
        review_count_info = review_count_tag.get_text(strip=True) if review_count_tag else "レビュー数なし"

        # 情報を保存
        hotels.append({
            "ホテル名": hotel_name,
            "詳細ページURL": detail_url,
            "アクセス情報": access_info,
            "価格情報": price_info,
            "一人あたり価格": unit_price_info,
            "ホテルの評価": rating_info,
            "レビュー数": review_count_info
        })
    
    return hotels

# 次のページURLを生成
def get_next_page_url(base_url, page_number):
    if page_number == 1:
        return base_url 
    else:
        return f"{base_url}page{page_number}/"

# 各URLについて処理
total_hotels_count = 0
for i, url in enumerate(urls, 1):
    print(f"URL {i}: {url}")
    all_hotels = []

    page_number = 1
    while True:
        current_url = get_next_page_url(url, page_number)
        print(f"現在のページ: {current_url}")
        hotels = get_hotel_data_from_page(current_url)
        
        if not hotels:  # ホテル情報が取得できなくなったら終了
            break
        
        all_hotels.extend(hotels)
        page_number += 1

    # URLごとのホテル数を表示
    print(f"URL {i} で取得したホテルの数: {len(all_hotels)}")
    total_hotels_count += len(all_hotels)

    # 結果を出力（必要に応じて省略可）
    for hotel in all_hotels:
        print(hotel)
        print("-" * 50)

# 総数を表示
print(f"全てのURLから取得したホテルの総数: {total_hotels_count}")


発見した a タグの数: 13
URL 1: https://www.jalan.net/130000/LRG_137100/
現在のページ: https://www.jalan.net/130000/LRG_137100/
現在のページ: https://www.jalan.net/130000/LRG_137100/page2/
URL 1 で取得したホテルの数: 32
{'ホテル名': '三交インＧｒａｎｄｅ東京浜松町', '詳細ページURL': "javascript:doYadDetailAd('376874','TGS01419110','115_1_1','2');", 'アクセス情報': 'JR・東京モノレール浜松町駅南口より徒歩約3分。都営大江戸線・浅草線「A1」出口より徒歩約4分。', '価格情報': '10,980円～', '一人あたり価格': '1名\xa05,490円～', 'ホテルの評価': '4.2', 'レビュー数': ''}
--------------------------------------------------
{'ホテル名': '第一ホテル東京', '詳細ページURL': "javascript:doYadDetailAd('385527','TGS01419110','115_1_2','2');", 'アクセス情報': 'JR・東京メトロ銀座線「新橋駅」より徒歩約2分/都営大江戸線「汐留駅」より徒歩約10分', '価格情報': '34,424円～', '一人あたり価格': '1名\xa017,212円～', 'ホテルの評価': '4.9', 'レビュー数': ''}
--------------------------------------------------
{'ホテル名': 'ラビスタ東京ベイ（共立リゾート）', '詳細ページURL': 'URLなし', 'アクセス情報': '新交通 ゆりかもめ「市場前駅」より徒歩約１分', '価格情報': '26,800円～', '一人あたり価格': '1名\xa013,400円～', 'ホテルの評価': '4.6', 'レビュー数': ''}
--------------------------------------------------
{'ホテル名': 'ハ

2025-01-24 01:04:38,818 - WARNING - Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


現在のページ: https://www.jalan.net/130000/LRG_137400/page2/
URL 4 で取得したホテルの数: 32
{'ホテル名': '�g�n�s�d�k�@�f���������P�i�z�e���@�Q�X�g�����j���w�O', '詳細ページURL': "javascript:doYadDetailAd('304424','TGS01419122','294_1_1','2');", 'アクセス情報': '�������g�����w����k��1���E�i�q���w����k��3��', '価格情報': '11,500�~�`', '一人あたり価格': '1��\xa05,750�~�`', 'ホテルの評価': '4.2', 'レビュー数': ''}
--------------------------------------------------
{'ホテル名': '�A�p�z�e�������]�[�g�q�����w�^���[�r', '詳細ページURL': "javascript:doYadDetailAd('346050','TGS01419122','294_1_2','2');", 'アクセス情報': 'JR�������u�����w�v�����k��3���^�s�c��]�ː��u�����w�vA3�o���k��4��', '価格情報': '9,600�~�`', '一人あたり価格': '1��\xa04,800�~�`', 'ホテルの評価': '4.2', 'レビュー数': ''}
--------------------------------------------------
{'ホテル名': '�z�e�����\\�����', '詳細ページURL': 'URLなし', 'アクセス情報': 'JR�u���w�v�\U000d140c����k��2���B�������g��������E����J���u���w�v���k��2���B', '価格情報': '18,000�~�`', '一人あたり価格': '1��\xa09,000�~�`', 'ホテルの評価': '4.6', 'レビュー数': ''}
--------------------------